<a href="https://colab.research.google.com/github/PokeMaster16/Machine-Learning-Projects/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
#import tensorflow as tf
import pandas as pd
#from tensorflow import keras
!pip install tensorflow-datasets
#import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

#print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# --- Load and preprocess ---
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=["label", "message"])

# Convert string labels to integers
label_map = {'ham': 0, 'spam': 1}
train_df['label'] = train_df['label'].map(label_map)
test_df['label'] = test_df['label'].map(label_map)

# --- Features and labels ---
X_train = train_df['message']
y_train = train_df['label']
X_test = test_df['message']
y_test = test_df['label']

# --- TF-IDF ---
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# --- Train model ---
model = LogisticRegression(class_weight='balanced')
model.fit(X_train_tfidf, y_train)

# --- Classifier function ---
def classify_message(message):
    msg_tfidf = vectorizer.transform([message])
    proba = model.predict_proba(msg_tfidf)[0]
    prediction = int(model.predict(msg_tfidf)[0])  # Now it's an integer (0 or 1)
    label = "Spam" if prediction == 1 else "Ham"
    confidence = proba[prediction]
    return f"{label} (Confidence: {confidence:.2f})"

# --- Try it ---
print(classify_message("Congratulations! You've won a free ticket. Call now!"))
print(classify_message("Hi, can we meet at 5 pm?"))


In [ ]:
def classify_message(message):
    msg_tfidf = vectorizer.transform([message])
    proba = model.predict_proba(msg_tfidf)[0]
    prediction = int(model.predict(msg_tfidf)[0])  # Make sure it's an integer
    label = "Spam" if prediction == 1 else "Ham"
    confidence = proba[prediction]  # Now safe
    return f"{label} (Confidence: {confidence:.2f})"

# Test
print(classify_message("Congratulations! You've won a free ticket. Call now!"))
print(classify_message("Hi, can we meet at 5 pm?"))


In [ ]:
def predict_message(pred_text):
    # Transform the message
    msg_tfidf = vectorizer.transform([pred_text])

    # Get probability of being spam (class 1)
    spam_prob = model.predict_proba(msg_tfidf)[0][1]  # Index 1 = probability of class 'spam'

    # Choose label based on threshold
    label = 'spam' if spam_prob >= 0.5 else 'ham'

    # Return result in required format
    return [spam_prob, label]

# Example usage
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


In [ ]:
import nbformat
from google.colab import _message

# Get current notebook as a raw dictionary
raw_nb = _message.blocking_request('get_ipynb')['ipynb']

# Convert raw dict to a proper nbformat NotebookNode object
notebook = nbformat.from_dict(raw_nb)

# Add missing 'name' field in kernelspec if it's missing
kspec = notebook['metadata'].get('kernelspec', {})
if 'name' not in kspec:
    kspec['name'] = 'python3'
    notebook['metadata']['kernelspec'] = kspec

# Save to new fixed notebook file
patched_filename = '/content/fixed_notebook.ipynb'

with open(patched_filename, 'w', encoding='utf-8') as f:
    nbformat.write(notebook, f)

print(f"✅ Patched notebook saved to: {patched_filename}")


In [ ]:
import nbformat

# Load the fixed notebook (replace the path if needed)
fixed_path = '/content/fixed_notebook.ipynb'

with open(fixed_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

print("Kernelspec metadata:")
print(nb.metadata.get('kernelspec'))

print("\nOther metadata:")
print(nb.metadata)



In [ ]:
import nbformat
from google.colab import _message

# Load current notebook raw dict
raw_nb = _message.blocking_request('get_ipynb')['ipynb']
notebook = nbformat.from_dict(raw_nb)

# Fix kernelspec metadata fully
kspec = notebook.metadata.get('kernelspec', {})
kspec.setdefault('name', 'python3')
kspec.setdefault('display_name', 'Python 3')
kspec.setdefault('language', 'python')
notebook.metadata['kernelspec'] = kspec

# Save fixed notebook
patched_filename = '/content/fixed_notebook_full_fix.ipynb'
with open(patched_filename, 'w', encoding='utf-8') as f:
    nbformat.write(notebook, f)

print(f"✅ Fully patched notebook saved to: {patched_filename}")


In [ ]:
import nbformat
from nbformat.validator import validate, ValidationError
from google.colab import _message

# Step 1: Load notebook from Colab frontend
raw_nb = _message.blocking_request('get_ipynb')['ipynb']
notebook = nbformat.from_dict(raw_nb)

# Step 2: Ensure required notebook-level metadata
notebook.nbformat = 4
notebook.nbformat_minor = 5

# Step 3: Fix kernelspec metadata
kspec = notebook.metadata.get('kernelspec', {})
kspec.setdefault('name', 'python3')
kspec.setdefault('display_name', 'Python 3')
kspec.setdefault('language', 'python')
notebook.metadata['kernelspec'] = kspec

# Step 4: Check/fix each cell
for i, cell in enumerate(notebook.cells):
    if 'metadata' not in cell:
        cell['metadata'] = {}
    if 'source' not in cell:
        cell['source'] = ''
    if 'cell_type' not in cell:
        print(f"⚠️ Cell {i} is missing 'cell_type'")
    if cell.get('cell_type') not in ['code', 'markdown', 'raw']:
        print(f"⚠️ Cell {i} has invalid cell_type: {cell.get('cell_type')}")

# Step 5: Try validation
try:
    validate(notebook)
    print("✅ Notebook passed validation.")
except ValidationError as e:
    print("❌ Notebook validation failed:")
    print(e)

# Step 6: Save the fully repaired notebook
patched_filename = '/content/fixed_notebook_validated.ipynb'
with open(patched_filename, 'w', encoding='utf-8') as f:
    nbformat.write(notebook, f)

print(f"📁 Saved validated notebook to: {patched_filename}")


In [ ]:
import nbformat
from nbformat.validator import validate
from google.colab import _message

# Load the current notebook from Colab
raw_nb = _message.blocking_request('get_ipynb')['ipynb']
notebook = nbformat.from_dict(raw_nb)

# Set nbformat version explicitly
notebook.nbformat = 4
notebook.nbformat_minor = 5

# === Fix top-level metadata ===
metadata = notebook.metadata

# Fix kernelspec
metadata['kernelspec'] = {
    "name": "python3",
    "display_name": "Python 3",
    "language": "python"
}

# Fix kernel_info
metadata['kernel_info'] = {
    "name": "python3"
}

# Fix language_info
metadata['language_info'] = {
    "name": "python",
    "version": "3.10.12",  # You can adjust this to match your environment
    "codemirror_mode": "python"
}

# Optional: add empty signature if not present
metadata.setdefault('signature', '')

# === Fix each cell ===
for cell in notebook.cells:
    cell.setdefault('metadata', {})
    cell.setdefault('source', '')
    if cell['cell_type'] == 'code':
        cell.setdefault('outputs', [])
        cell.setdefault('execution_count', None)

# === Validate ===
try:
    validate(notebook)
    print("✅ Notebook is fully valid.")
except Exception as e:
    print("❌ Validation failed:")
    print(e)

# === Save notebook ===
output_path = '/content/fixed_notebook_fullschema.ipynb'
with open(output_path, 'w', encoding='utf-8') as f:
    nbformat.write(notebook, f)

print(f"📁 Fully fixed notebook saved to: {output_path}")


In [ ]:
import nbformat
from google.colab import _message

# 1. Get current notebook contents
raw_nb = _message.blocking_request('get_ipynb')['ipynb']
old_nb = nbformat.from_dict(raw_nb)

# 2. Create a brand new, clean notebook
new_nb = nbformat.v4.new_notebook()

# 3. Copy all cells from the old notebook
new_nb.cells = old_nb.cells

# 4. Set proper metadata
new_nb.metadata = {
    "kernelspec": {
        "name": "python3",
        "display_name": "Python 3",
        "language": "python"
    },
    "language_info": {
        "name": "python",
        "version": "3.10.12",
        "codemirror_mode": "python"
    }
}

new_nb.nbformat = 4
new_nb.nbformat_minor = 5

# 5. Save the new notebook
output_path = "/content/rebuilt_notebook.ipynb"
with open(output_path, "w", encoding="utf-8") as f:
    nbformat.write(new_nb, f)

print(f"✅ Rebuilt notebook saved to: {output_path}")
